## Part II - Train and evaluate the model

####  Required Python libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing an ipynb file from another ipynb file
!pip install ipynb

In [3]:
# Importing functions from another jupyter notebook
!pip install nbimporter

In [4]:
%run GlobalConfig.ipynb

Setting global variables...


In [5]:
import nbimporter
import loader_nb

loader = loader_nb.Loader_nb()

init Loader notebook


In [6]:
help(loader.load_data)

Help on method load_data in module loader_nb:

load_data(url='https://raw.githubusercontent.com/quickheaven/scs-3253-machine-learning/master/datasets/ISCX-URL2016_All.csv') method of loader_nb.Loader_nb instance
    (string) --> dataframe
    
    This function returns the dataframe of maliciours url.    
    
    Parameters
    ----------
    url: By default, it fetch the data from github otherwise a local path or url can be provided to load the data frame faster.



In [7]:
help(loader.prepare_data)

Help on method prepare_data in module loader_nb:

prepare_data(data, fill_na=True, feature_selection=True) method of loader_nb.Loader_nb instance
    (DataFrame, boolean, boolean) --> X and y of the dataframe.
    
    This function returns the X and y of the malicious url dataframe.
    
    Parameters
    ----------
    fill_na : True to fill the na records with mean values otherwise drop the features.
    
    feature_selection : True to remove one or more features that have a correlation higher than 0.9 othewise do not perform that kind of feature selection.

